In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import cv2
import numpy as np
import random
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)         
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
data_dir = '/content/gdrive/MyDrive/tugas-akhir-dhifaf/dataset-ta/all-aug-ex-1-4/A/img-A-9307.jpg'

In [ ]:
# Loading the Tensorflow Saved Model (PB)
h5_model = "./mymodel.h5"
model = tf.keras.models.load_model('/content/gdrive/MyDrive/tugas-akhir-dhifaf/new-model-ex4-best-model-47466/best_model')
print(model.summary())

# # Saving the Model in H5 Format
# tf.keras.models.save_model(model, h5_model)

# # Loading the H5 Saved Model
# loaded_model_from_h5 = tf.keras.models.load_model(h5_model)
# print(loaded_model_from_h5.summary())

Model: "model_1_ex2_fix"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_4 (Rescaling)        (None, 256, 256, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv2d_24 (Conv2D)             (None, 128, 128, 32  896         ['rescaling_4[0][0]']            
                                )                                                                 
                                                                                    

In [ ]:
# get list of layers on model
layer_outputs = [layer.output for layer in model.layers[1:]]

In [ ]:
# make a visualization of a model
visualize_model = tf.keras.models.Model(inputs = model.input, outputs = layer_outputs)

In [ ]:
for layer_name, feature_map in zip(layer_names, feature_maps):print(f"The shape of the {layer_name} is =======>> {feature_map.shape}")

The shape of the input_5 is =======>> (1, 256, 256, 3)
The shape of the rescaling_4 is =======>> (1, 128, 128, 32)
The shape of the conv2d_24 is =======>> (1, 64, 64, 32)
The shape of the max_pooling2d_24 is =======>> (1, 64, 64, 32)
The shape of the batch_normalization_52 is =======>> (1, 64, 64, 32)
The shape of the activation_48 is =======>> (1, 64, 64, 64)
The shape of the conv2d_25 is =======>> (1, 64, 64, 64)
The shape of the batch_normalization_53 is =======>> (1, 32, 32, 64)
The shape of the max_pooling2d_25 is =======>> (1, 32, 32, 64)
The shape of the activation_49 is =======>> (1, 32, 32, 128)
The shape of the separable_conv2d_44 is =======>> (1, 32, 32, 128)
The shape of the batch_normalization_54 is =======>> (1, 16, 16, 128)
The shape of the max_pooling2d_26 is =======>> (1, 16, 16, 128)
The shape of the conv2d_26 is =======>> (1, 16, 16, 128)
The shape of the add_24 is =======>> (1, 16, 16, 128)
The shape of the activation_50 is =======>> (1, 16, 16, 256)
The shape of th

In [ ]:
img = load_img(data_dir, target_size=(256,256))

In [ ]:
x = img_to_array(img)
x.shape

(256, 256, 3)

In [ ]:
x = x.reshape((1,256,256,3))
print(x.shape)

(1, 256, 256, 3)


In [ ]:
x = x / 255

In [ ]:
feature_maps = visualize_model.predict(x)
print(len(feature_maps))

1/1 [==============================] - 0s 107ms/step
58


In [ ]:
layer_names = [layer.name for layer in model.layers]
print(layer_names)

['input_5', 'rescaling_4', 'conv2d_24', 'max_pooling2d_24', 'batch_normalization_52', 'activation_48', 'conv2d_25', 'batch_normalization_53', 'max_pooling2d_25', 'activation_49', 'separable_conv2d_44', 'batch_normalization_54', 'max_pooling2d_26', 'conv2d_26', 'add_24', 'activation_50', 'separable_conv2d_45', 'batch_normalization_55', 'max_pooling2d_27', 'conv2d_27', 'add_25', 'activation_51', 'separable_conv2d_46', 'batch_normalization_56', 'max_pooling2d_28', 'conv2d_28', 'add_26', 'activation_52', 'separable_conv2d_47', 'batch_normalization_57', 'activation_53', 'separable_conv2d_48', 'batch_normalization_58', 'activation_54', 'separable_conv2d_49', 'batch_normalization_59', 'add_27', 'activation_55', 'separable_conv2d_50', 'batch_normalization_60', 'activation_56', 'separable_conv2d_51', 'batch_normalization_61', 'activation_57', 'separable_conv2d_52', 'batch_normalization_62', 'add_28', 'activation_58', 'separable_conv2d_53', 'batch_normalization_63', 'max_pooling2d_29', 'conv2d_2

In [ ]:
plt.rcParams.update({'font.size': 72})

In [ ]:
#plotting 
for layer_names, feature_maps in zip(layer_names, feature_maps):
  print(feature_maps.shape)
  if len(feature_maps.shape) == 4 :
    channels = feature_maps.shape[-1]
    size = feature_maps.shape[1]
    display_grid = np.zeros((size, size * channels))
    for i in range(channels):
      x = feature_maps[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x

    scale = 200. / channels
    plt.figure(figsize=(scale * channels, scale))
    plt.title(layer_names)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    file_dir = 'feature-map/'
    file_name = f'feature-map-plot-{layer_names}.png'
    plt.savefig(file_dir+file_name)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
plt.rcParams.update({'font.size': 24})

In [ ]:
#plotting 
img = load_img(data_dir, target_size=(256,256))
x = img_to_array(img)
x.shape
x = x.reshape((1,256,256,3))
x = x / 255
feature_maps = visualize_model.predict(x)
layer_names = [layer.name for layer in model.layers]
for layer_names, feature_maps in zip(layer_names, feature_maps):
  print(feature_maps.shape)
  if len(feature_maps.shape) == 4 :
    channels = feature_maps.shape[-1]
    size = feature_maps.shape[1]
    display_grid = np.zeros((size, size * channels))
    for i in range(channels):
      x = feature_maps[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x

    scale = 30. / channels
    plt.figure(figsize=(scale * channels, scale))
    plt.title(layer_names)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    file_dir = 'feature-map-small-ver/'
    file_name = f'feature-map-plot-{layer_names}.png'
    plt.savefig(file_dir+file_name)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!zip -r feature-map.zip {'/content/feature-map'}

  adding: content/feature-map/ (stored 0%)
  adding: content/feature-map/feature-map-plot-max_pooling2d_26.png (deflated 9%)
  adding: content/feature-map/feature-map-plot-activation_48.png (deflated 2%)
  adding: content/feature-map/feature-map-plot-conv2d_25.png (deflated 2%)
  adding: content/feature-map/feature-map-plot-max_pooling2d_28.png (deflated 7%)
  adding: content/feature-map/feature-map-plot-conv2d_28.png (deflated 7%)
  adding: content/feature-map/feature-map-plot-batch_normalization_64.png (deflated 4%)
  adding: content/feature-map/feature-map-plot-conv2d_26.png (deflated 10%)
  adding: content/feature-map/feature-map-plot-batch_normalization_60.png (deflated 7%)
  adding: content/feature-map/feature-map-plot-batch_normalization_53.png (deflated 10%)
  adding: content/feature-map/feature-map-plot-add_27.png (deflated 6%)
  adding: content/feature-map/feature-map-plot-separable_conv2d_49.png (deflated 5%)
  adding: content/feature-map/feature-map-plot-add_24.png (deflate

In [ ]:
!cp /content/feature-map.zip gdrive/MyDrive/tugas-akhir-dhifaf/

In [ ]:

!zip -r feature-map-small-ver.zip {'/content/feature-map-small-ver'}

  adding: content/feature-map-small-ver/ (stored 0%)
  adding: content/feature-map-small-ver/feature-map-plot-max_pooling2d_26.png (deflated 0%)
  adding: content/feature-map-small-ver/feature-map-plot-activation_48.png (deflated 1%)
  adding: content/feature-map-small-ver/feature-map-plot-conv2d_25.png (deflated 1%)
  adding: content/feature-map-small-ver/feature-map-plot-max_pooling2d_28.png (deflated 0%)
  adding: content/feature-map-small-ver/feature-map-plot-conv2d_28.png (deflated 0%)
  adding: content/feature-map-small-ver/feature-map-plot-batch_normalization_64.png (stored 0%)
  adding: content/feature-map-small-ver/feature-map-plot-conv2d_26.png (deflated 0%)
  adding: content/feature-map-small-ver/feature-map-plot-batch_normalization_60.png (stored 0%)
  adding: content/feature-map-small-ver/feature-map-plot-batch_normalization_53.png (deflated 0%)
  adding: content/feature-map-small-ver/feature-map-plot-add_27.png (deflated 0%)
  adding: content/feature-map-small-ver/feature

In [ ]:
!cp /content/feature-map-small-ver.zip gdrive/MyDrive/tugas-akhir-dhifaf/

In [ ]:
!python --version

Python 3.8.10


In [ ]:
pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.4
aiosignal                     1.3.1
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        6.0.0
blis  